# Multi Task Model

## Imports

In [1]:
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNet
import numpy as np

from loadData_alt import createDataset

## Model parameters

In [2]:
# Log parameters
model_name = 'multiTaskModel_alt2'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
doFineTuning = True
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 10

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    # keras.callbacks.EarlyStopping(
    #                 monitor="val_loss",
    #                 patience=2,
    #                 verbose=1)
]

## Data parameters

In [4]:
image_height = 160
image_width = 160

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [6]:
training_ds, validation_ds = createDataset('../../data/m3/training', batch_size, 0.2)

ValueError: in user code:

    File "e:\Master Studium\Computer Vision\Projekt\dl_faceRecognition\src\m3\loadData_alt.py", line 61, in process_path  *
        label = get_label(file_path)
    File "e:\Master Studium\Computer Vision\Projekt\dl_faceRecognition\src\m3\loadData_alt.py", line 17, in get_label  *
        filepath_split = tf.strings.split(file_path, '/')

    ValueError: input: Tensor conversion requested dtype string for Tensor with dtype float32: <tf.Tensor 'args_0:0' shape=() dtype=float32>


## Creating the test dataset

In [ ]:
#test_ds = createDataset('../../data/m3/training', batch_size, 0)

## Load the ResNet model

In [ ]:
# Loading either the MobileNet architecture model or the previously saved model, and freeze it for transfer learning
mobilenet = MobileNet(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                input_tensor=None,
                pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
mobilenet.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))
# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = mobilenet(inputs, training=False)

base_model = keras.layers.GlobalAveragePooling2D()(base_model)

## Creating Task 1 (Face Detection) Top Model

In [ ]:
# Add Dense layer
face_head = tf.keras.layers.Dense(256)(base_model)
face_head = tf.keras.layers.Dense(128)(face_head)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(face_head)


## Creating Task 2 (Mask Detection) Top Model

In [ ]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(256)(base_model)
mask_head = tf.keras.layers.Dense(128)(mask_head)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [ ]:
# Add Dense layer
age_head = tf.keras.layers.Dense(512)(base_model)
age_head = tf.keras.layers.Dense(256)(age_head)
age_head = tf.keras.layers.Dense(128)(age_head)

# Final layer for binary classification
age_outputs = keras.layers.Dense(122, activation='softmax', name='age_output')(age_head)

## Creating and compiling the final model

In [ ]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])
keras.utils.plot_model(model)

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 1

model.compile(
      optimizer=keras.optimizers.Adam(0.001), # Learning Rate?
            loss={
                  'face_output': keras.losses.BinaryCrossentropy(), 
                  'mask_output': keras.losses.BinaryCrossentropy(),
                  'age_output': keras.losses.SparseCategoricalCrossentropy()
                  },
            loss_weights={
                  'face_output': 0.25 * gamma, 
                  'mask_output': 0.25 * gamma,
                  'age_output': 0.5 * gamma
                  }, 
            metrics={
                  'face_output': keras.metrics.BinaryAccuracy(), 
                  'mask_output': keras.metrics.BinaryAccuracy(), 
                  'age_output': keras.metrics.SparseCategoricalAccuracy()
                  },
)

model.summary()

## Training the model with the dataset

In [ ]:
history = model.fit(
            training_ds,
            validation_data=validation_ds,
            epochs=epochs, 
            callbacks=callbacks
        )

## Fine tuning

In [ ]:
if doFineTuning:
      model_name = model_name + '_ft'
      mobilenet.trainable = True
      model.summary()

      model.compile(
            optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
            loss={
                  'face_output': keras.losses.BinaryCrossentropy(), 
                  'mask_output': keras.losses.BinaryCrossentropy(),
                  'age_output': keras.losses.SparseCategoricalCrossentropy()
                  },
            loss_weights={
                  'face_output': gamma, 
                  'mask_output': gamma,
                  'age_output': gamma
                  }, 
                  metrics=['accuracy']
      )

      model.fit(training_ds, epochs=epochs)

## Save the model

In [ ]:
model.save(savedModelPath)

## Test model with test dataset

In [ ]:
results = model.evaluate(test_ds)

print(f'Loss: {results[0]}; Accuracy: {results[1]}')

## Predict new images

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../../data/m3/training/face/mask/1_1_-1_400066.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

preds = model.predict(img_array)

print(f"Face: {preds[0][0]*100}%")
print(f"Mask: {preds[1][0]*100}%")
print(f"Age: {np.argmax(preds[2][0])}")